In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import torch
print("pytorch ver . ",torch.__version__)
print("Is CUDA available?",torch.cuda.is_available())
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
import torch.utils.data as Data
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import math

/home/luhung3080/miniconda3/envs/chou/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


pytorch ver .  1.11.0+cu113
Is CUDA available? True


# Data

In [2]:
data=pd.read_csv('/home/luhung3080/Desktop/PycharmProjects/NCHUproject/Transformer/data_final.csv')
data

,SITE_ID,date,FCST_TIME,TAU,pm25_cal,pm25_obs
0,EPA001,2020-02-24 08:00:00,2020-02-24 09:00:00,1,4.9510,10.0
1,EPA001,2020-02-24 08:00:00,2020-02-24 10:00:00,2,4.4674,13.0
2,EPA001,2020-02-24 08:00:00,2020-02-24 11:00:00,3,4.6159,11.0
3,EPA001,2020-02-24 08:00:00,2020-02-24 12:00:00,4,3.9937,11.0
4,EPA001,2020-02-24 08:00:00,2020-02-24 13:00:00,5,3.9602,9.0
...,...,...,...,...,...,...
3092755,EPA080,2021-10-30 08:00:00,2021-11-02 04:00:00,68,3.6190,4.0
3092756,EPA080,2021-10-30 08:00:00,2021-11-02 05:00:00,69,3.7908,7.0
3092757,EPA080,2021-10-30 08:00:00,2021-11-02 06:00:00,70,4.0454,7.0
3092758,EPA080,2021-10-30 08:00:00,2021-11-02 07:00:00,71,3.9015,4.0


In [3]:
x1=np.zeros([604,1704])
x2=np.zeros([604,5112])

#x1 (0th~1703th column as x)
for i in range (0,604):
    for j in range (0,71):
        a=np.array(data['pm25_obs'][5112*i+72*j:5112*i+72*j+24])
        for k in range (0,24):
            x1[i][j*24+k]=a[k]

#x2 (1704th~8519th column as x)
for i in range (1,604):
    b=np.array(data['pm25_cal'][5112*i:5112*i+5112])
    for j in range(0,5112):
        x2[i-1][j]=b[j]
        
print(np.shape(x1))
print(np.shape(x2))

(604, 1704)
(604, 5112)


In [4]:
x1Restruct_Fun=x1
x2Restruct_Fun=x2
print(np.shape(x1Restruct_Fun))
print(np.shape(x2Restruct_Fun))

(604, 1704)
(604, 5112)


In [5]:
YRestruct_Fun=np.zeros([601,5112])
for j in range (0,71):
    for i in range(0,601):
        YRestruct_Fun[i][72*j:72*j+24]=x1Restruct_Fun[1+i][24*j:24*j+24]
        YRestruct_Fun[i][72*j+24:72*j+48]=x1Restruct_Fun[1+i+1][24*j:24*j+24]
        YRestruct_Fun[i][72*j+48:72*j+72]=x1Restruct_Fun[1+i+2][24*j:24*j+24]
print(np.shape(YRestruct_Fun))

(601, 5112)


In [6]:
XRestruct_Fun=np.zeros([601,6816])
for i in range (0,601):
    for j in range (0,1704):
        XRestruct_Fun[i][j]=x1Restruct_Fun[i][j]
    for j in range (1704,6816):
        XRestruct_Fun[i][j]=x2Restruct_Fun[i][j-1704]
print(np.shape(XRestruct_Fun))

(601, 6816)


In [7]:
Xhat=XRestruct_Fun
Yhat=YRestruct_Fun
Xhat_train = np.zeros([540,6816])
Yhat_train = np.zeros([540,5112])
Xhat_val = np.zeros([61,6816])
Yhat_val = np.zeros([61,5112])
Xhat_test = np.zeros([61,6816])
Yhat_test = np.zeros([61,5112])

for i in range (0,540):
    for j in range (0,1704):
        Xhat_train[i][j] = Xhat[i][j]
    for j in range (0,5112):    
        Xhat_train[i][j+1704] = Xhat[i][j+1704]
        Yhat_train[i][j] = Yhat[i][j]
        
for i in range (540,601):
    for j in range (0,1704):
        Xhat_val[i-540][j] = Xhat[i][j]
    for j in range (0,5112):
        Xhat_val[i-540][j+1704] = Xhat[i][j+1704]
        Yhat_val[i-540][j] = Yhat[i][j]     
        
for i in range (540,601):
    for j in range (0,1704):
        Xhat_test[i-540][j] = Xhat[i][j]
    for j in range (0,5112):
        Xhat_test[i-540][j+1704] = Xhat[i][j+1704]
        Yhat_test[i-540][j] = Yhat[i][j]
        
print('Xhat_train')
print(np.shape(Xhat_train))
print('Yhat_train')
print(np.shape(Yhat_train))
print('Xhat_val')
print(np.shape(Xhat_val))
print('Yhat_val')
print(np.shape(Yhat_val))
print('Xhat_test')
print(np.shape(Xhat_test))
print('Yhat_test')
print(np.shape(Yhat_test))

Xhat_train
(540, 6816)
Yhat_train
(540, 5112)
Xhat_val
(61, 6816)
Yhat_val
(61, 5112)
Xhat_test
(61, 6816)
Yhat_test
(61, 5112)


In [8]:
Xhat_train_n = np.zeros([71,540,96])
Yhat_train_n = np.zeros([71,540,72])
Xhat_test_n = np.zeros([71,61,96])
Yhat_test_n = np.zeros([71,61,72])
for i in range (0,71):
    for j in range (0,540):
        for k in range (0,24):
            Xhat_train_n[i][j][k] = Xhat_train[j][24*i+k]
        for k in range (0,72):
            Xhat_train_n[i][j][k+24] = Xhat_train[j][1704+(72*i)+k]
            Yhat_train_n[i][j][k] = Yhat_train[j][(72*i)+k]
    for j in range (0,61):
        for k in range (0,24):
            Xhat_test_n[i][j][k] = Xhat_test[j][24*i+k]
        for k in range (0,72):
            Xhat_test_n[i][j][k+24] = Xhat_test[j][1704+(72*i)+k]
            Yhat_test_n[i][j][k] = Yhat_test[j][(72*i)+k]
print('Xhat_train')
print(np.shape(Xhat_train_n))
print('Yhat_train')
print(np.shape(Yhat_train_n))
print('Xhat_test')
print(np.shape(Xhat_test_n))
print('Yhat_test')
print(np.shape(Yhat_test_n))
Xhat_val_n=Xhat_test_n
Yhat_val_n=Yhat_test_n
print('Xhat_val')
print(np.shape(Xhat_val_n))
print('Yhat_val')
print(np.shape(Yhat_val_n))

Xhat_train
(71, 540, 96)
Yhat_train
(71, 540, 72)
Xhat_test
(71, 61, 96)
Yhat_test
(71, 61, 72)
Xhat_val
(71, 61, 96)
Yhat_val
(71, 61, 72)


In [9]:
Xhat_train_n0 = np.zeros([71, 540, 168])
Xhat_val_n0 = np.zeros([71, 61, 168])
Xhat_test_n0 = np.zeros([71, 61, 168])
for k in range(0,71):
    for i in range (0,540):
        for j in range (0,96):
            Xhat_train_n0[k][i][j] = Xhat_train_n[k][i][j]
        for j in range (96,168):
            Xhat_train_n0[k][i][j] = Yhat_train_n[k][i][j-96]
for k in range(0,71):
    for i in range (0,61):
        for j in range (0,96):
            Xhat_val_n0[k][i][j] = Xhat_val_n[k][i][j]
        for j in range (96,168):
            Xhat_val_n0[k][i][j] = Yhat_val_n[k][i][j-96]
for k in range(0,71):
    for i in range (0,61):
        for j in range (0,96):
            Xhat_test_n0[k][i][j] = Xhat_test_n[k][i][j]
        for j in range (96,168):
            Xhat_test_n0[k][i][j] = Yhat_test_n[k][i][j-96]
print('Xhat_train')
print(np.shape(Xhat_train_n0))
print('Yhat_train')
print(np.shape(Yhat_train_n))
print('Xhat_test')
print(np.shape(Xhat_test_n0))
print('Yhat_test')
print(np.shape(Yhat_test_n))
print('Xhat_val')
print(np.shape(Xhat_val_n0))
print('Yhat_val')
print(np.shape(Yhat_val_n))

Xhat_train
(71, 540, 168)
Yhat_train
(71, 540, 72)
Xhat_test
(71, 61, 168)
Yhat_test
(71, 61, 72)
Xhat_val
(71, 61, 168)
Yhat_val
(71, 61, 72)


# Modeling

In [10]:
x_train = torch.from_numpy(Xhat_train_n0[0]).float()
y_train = torch.from_numpy(Yhat_train_n[0]).float()
x_test = torch.from_numpy(Xhat_test_n0[0]).float()
y_test = torch.from_numpy(Yhat_test_n[0]).float()
x_val = torch.from_numpy(Xhat_val_n0[0]).float()
y_val = torch.from_numpy(Yhat_val_n[0]).float()

In [12]:
class PytorchLightningModel(pl.LightningModule): # 一定要繼承pl.LightningModule
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(96,72)
        self.layer2 = nn.Transformer(d_model=72, nhead=72)
    
    def forward(self, x):
        ouput = self.layer1(x[:,:96])
        ouput = self.layer2(ouput, x[:,96:]) 
        return ouput

    def configure_optimizers(self): # 自動訓練時會呼叫此方法來獲取Optimizer.
        return optim.Adam(self.parameters(), lr=1e-3) # 這邊注意要調整的參數是`self.parameters()`

    def prepare_data(self): # 此方法會在初始化後優先執行。 所以可以在此方法中先將會用到的資料都讀取進來.
        self.train_set = Data.TensorDataset(x_train , y_train) 
        self.test_set = Data.TensorDataset(x_test, y_test)
        self.val_set = Data.TensorDataset(x_val, y_val)

    # 以下三個方法則是設定進行訓練及驗證時所要使用的Data Loader格式。
    def train_dataloader(self):
        # return Data.DataLoader(dataset=self.train_set, batch_size= self.batch_size, shuffle=True)
        return Data.DataLoader(dataset=self.train_set , batch_size=540 , shuffle=True)

    def test_dataloader(self):
        # return Data.DataLoader(dataset=self.test_set, batch_size= self.batch_size, shuffle=True)
        return Data.DataLoader(dataset=self.test_set , batch_size=61 , shuffle=False)

    def val_dataloader(self):
        #return Data.DataLoader(dataset=self.val_set, batch_size= self.batch_size, shuffle=True)
        return Data.DataLoader(dataset=self.val_set , batch_size=61 , shuffle=False)

#----------------------------------------------------------------------------------------------------------------------------------------------

    def training_step(self, batch, batch_idx): # 定義訓練過程的Step要如何進行
        x, y = batch # 從self.train_dataloader()的Data Loader取一個batch出來。
        output  = self.forward(x)
        loss = nn.L1Loss(reduction='mean')(output, y)
        logs = {'loss' : loss }
        self.log("train_loss_MAE", loss)
        return {'loss' : loss ,'log' : logs }

    def test_step(self, batch, batch_idx): #定義 Test 階段
        x, y = batch 
        output  = self.forward(x)
        loss = nn.L1Loss(reduction='mean')(output, y)
        logs = {'loss' : loss }
        self.log("test_loss_MAE", loss)
        return {'loss' : loss ,'log' : logs}

    # def test_eposh_end(self, outputs):
    #     avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
    #     logs = {'test_loss': avg_loss}      
    #     #return {'avg_test_loss': avg_loss, 'log': logs, 'progress_bar': logs }
    #     return {'avg_test_loss': avg_loss, 'log': logs}

    def validation_step(self, batch, batch_idx): # 定義Validation如何進行，以這邊為例就再加上了計算Acc.
        x, y = batch 
        output  = self.forward(x)
        loss = nn.L1Loss(reduction='mean')(output, y)
        logs = {'loss' : loss }
        self.log("val_loss_MAE", loss)
        return {'loss' : loss ,  'log' : logs}
    
    # def validation_epoch_end(self, outputs): # 在Validation的一個Epoch結束後，計算平均的Loss.
    #     avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
    #     logs = {'val_loss': avg_loss}
    #     return {'avg_val_loss': avg_loss, 'log' : logs, 'progress_bar': logs}

In [13]:
logger1 = TensorBoardLogger("jupyterNB_logs", name="Transformer")
logger2 = CSVLogger("jupyterNB_logs", name="Transformer_csv")
model = PytorchLightningModel() 
trainer = pl.Trainer(max_epochs=500 , gpus=1 , logger=logger1)
trainer.fit(model)
trainer.test(model)
trainer.test(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type        | Params
---------------------------------------
0 | layer1 | Linear      | 7.0 K 
1 | layer2 | Transformer | 3.9 M 
---------------------------------------
4.0 M     Trainable params
0         Non-trainable params
4.0 M     Total params
15.818    Total estimated model params size (MB)


/home/luhung3080/miniconda3/envs/chou/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/home/luhung3080/miniconda3/envs/chou/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/home/luhung3080/miniconda3/envs/chou/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1942: PossibleUserWarning: The number

Epoch 0:  50%|████████        | 1/2 [00:00<00:00, 15.41it/s, loss=11.8, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 1:  50%|████████        | 1/2 [00:00<00:00,  3.96it/s, loss=11.8, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 2:  50%|████████        | 1/2 [00:00<00:00,  2.31it/s, loss=11.8, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 3:  50%|████████        | 1/2 [00:00<00:00,  1.63it/s, loss=11.8, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 4:  50%|████████        | 1/2 [00:00<00:00,  1.25it/s, loss=11.8, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 5:  50%|████████        | 1/2 [00:00<00:00,  1.03it/s, loss=11.8, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 6:  50%|████████        | 1/2 [00:01<00:01,  1.15s/it, loss=11.8, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 7:  50%|████████        | 1/2 [00:01<00:01,  1.33s/it, loss=11.8, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 8:  50%|████████        | 1/2 [00:01<00:01,  1.51s/it, loss=11.8, v_num=2]
Validation: 0it [00:00,

Epoch 29:  50%|███████▌       | 1/2 [00:05<00:05,  5.24s/it, loss=11.8, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 30:  50%|███████▌       | 1/2 [00:05<00:05,  5.41s/it, loss=11.8, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 31:  50%|███████▌       | 1/2 [00:05<00:05,  5.58s/it, loss=11.8, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 32:  50%|███████▌       | 1/2 [00:05<00:05,  5.75s/it, loss=11.8, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 33:  50%|███████▌       | 1/2 [00:05<00:05,  5.93s/it, loss=11.7, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 34:  50%|███████▌       | 1/2 [00:06<00:06,  6.10s/it, loss=11.7, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 35:  50%|███████▌       | 1/2 [00:06<00:06,  6.27s/it, loss=11.7, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 36:  50%|███████▌       | 1/2 [00:06<00:06,  6.45s/it, loss=11.7, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 37:  50%|███████▌       | 1/2 [00:06<00:06,  6.63s/it, loss=11.7, v_num=2]
Validation: 0it [00:00,

Epoch 57:  50%|███████▌       | 1/2 [00:10<00:10, 10.21s/it, loss=11.7, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 58:  50%|███████▌       | 1/2 [00:10<00:10, 10.38s/it, loss=11.7, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 59:  50%|███████▌       | 1/2 [00:10<00:10, 10.55s/it, loss=11.7, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 60:  50%|███████▌       | 1/2 [00:10<00:10, 10.72s/it, loss=11.7, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 61:  50%|███████▌       | 1/2 [00:10<00:10, 10.90s/it, loss=11.7, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 62:  50%|███████▌       | 1/2 [00:11<00:11, 11.08s/it, loss=11.7, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 63:  50%|███████▌       | 1/2 [00:11<00:11, 11.25s/it, loss=11.7, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 64:  50%|███████▌       | 1/2 [00:11<00:11, 11.42s/it, loss=11.7, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 65:  50%|███████▌       | 1/2 [00:11<00:11, 11.59s/it, loss=11.7, v_num=2]
Validation: 0it [00:00,

...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/luhung3080/miniconda3/envs/chou/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Testing DataLoader 0: 100%|███████████████████████| 1/1 [00:00<00:00, 85.03it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_loss_MAE          7.942449569702148
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|███████████████████████| 1/1 [00:00<00:00, 88.99it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_loss_MAE          7.942449569702148
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss_MAE': 7.942449569702148}]

In [14]:
model_test = PytorchLightningModel.load_from_checkpoint('/home/luhung3080/Desktop/PycharmProjects/NCHUproject/Transformer/jupyterNB_logs/Transformer/version_2/checkpoints/epoch=499-step=500.ckpt')
trainer = pl.Trainer()  
trainer.test(model_test)

/home/luhung3080/miniconda3/envs/chou/lib/python3.7/site-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  category=PossibleUserWarning,
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/luhung3080/miniconda3/envs/chou/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1826: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  category=PossibleUserWarning,


Testing DataLoader 0: 100%|███████████████████████| 1/1 [00:00<00:00, 53.83it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_loss_MAE          7.942445278167725
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss_MAE': 7.942445278167725}]

In [ ]:
for i in range (0,71):
    x_train = torch.from_numpy(Xhat_train_n0[i]).float()
    y_train = torch.from_numpy(Yhat_train_n[i]).float()
    x_test = torch.from_numpy(Xhat_test_n0[i]).float()
    y_test = torch.from_numpy(Yhat_test_n[i]).float()
    x_val = torch.from_numpy(Xhat_val_n0[i]).float()
    y_val = torch.from_numpy(Yhat_val_n[i]).float()
    
    logger = CSVLogger("jupyterNB_logs", name="Transformer_csv")
    model = PytorchLightningModel() 
    trainer = pl.Trainer(max_epochs=500 , gpus=1 , logger=logger)
    trainer.fit(model)
    trainer.test(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: jupyterNB_logs/Transformer_csv
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type        | Params
---------------------------------------
0 | layer1 | Linear      | 7.0 K 
1 | layer2 | Transformer | 3.9 M 
---------------------------------------
4.0 M     Trainable params
0         Non-trainable params
4.0 M     Total params
15.818    Total estimated model params size (MB)


Epoch 0:  50%|████████        | 1/2 [00:00<00:00, 16.18it/s, loss=11.8, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 1:  50%|████████        | 1/2 [00:00<00:00,  4.26it/s, loss=11.8, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 2:  50%|████████        | 1/2 [00:00<00:00,  2.22it/s, loss=11.8, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 3:  50%|████████        | 1/2 [00:00<00:00,  1.62it/s, loss=11.8, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 4:  50%|████████        | 1/2 [00:00<00:00,  1.27it/s, loss=11.8, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 5:  50%|████████        | 1/2 [00:00<00:00,  1.04it/s, loss=11.8, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 6:  50%|████████        | 1/2 [00:01<00:01,  1.13s/it, loss=11.8, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 7:  50%|████████        | 1/2 [00:01<00:01,  1.30s/it, loss=11.8, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 8:  50%|████████        | 1/2 [00:01<00:01,  1.47s/it, loss=11.8, v_num=0]
Validation: 0it [00:00,

Epoch 29:  50%|███████▌       | 1/2 [00:05<00:05,  5.13s/it, loss=11.8, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 30:  50%|███████▌       | 1/2 [00:05<00:05,  5.29s/it, loss=11.8, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 31:  50%|███████▌       | 1/2 [00:05<00:05,  5.46s/it, loss=11.8, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 32:  50%|███████▌       | 1/2 [00:05<00:05,  5.62s/it, loss=11.8, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 33:  50%|███████▌       | 1/2 [00:05<00:05,  5.79s/it, loss=11.7, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 34:  50%|███████▌       | 1/2 [00:05<00:05,  5.95s/it, loss=11.7, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 35:  50%|███████▌       | 1/2 [00:06<00:06,  6.12s/it, loss=11.7, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 36:  50%|███████▌       | 1/2 [00:06<00:06,  6.29s/it, loss=11.7, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 37:  50%|███████▌       | 1/2 [00:06<00:06,  6.46s/it, loss=11.7, v_num=0]
Validation: 0it [00:00,

Epoch 57:  50%|███████▌       | 1/2 [00:09<00:09,  9.80s/it, loss=11.7, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 58:  50%|███████▌       | 1/2 [00:09<00:09,  9.96s/it, loss=11.7, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 59:  50%|███████▌       | 1/2 [00:10<00:10, 10.13s/it, loss=11.7, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 60:  50%|███████▌       | 1/2 [00:10<00:10, 10.29s/it, loss=11.7, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 61:  50%|███████▌       | 1/2 [00:10<00:10, 10.46s/it, loss=11.7, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 62:  50%|███████▌       | 1/2 [00:10<00:10, 10.67s/it, loss=11.7, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 63:  50%|███████▌       | 1/2 [00:10<00:10, 10.85s/it, loss=11.7, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 64:  50%|███████▌       | 1/2 [00:11<00:11, 11.02s/it, loss=11.7, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 65:  50%|███████▌       | 1/2 [00:11<00:11, 11.19s/it, loss=11.7, v_num=0]
Validation: 0it [00:00,

Epoch 86:  50%|███████▌       | 1/2 [00:14<00:14, 14.65s/it, loss=11.6, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 87:  50%|███████▌       | 1/2 [00:14<00:14, 14.81s/it, loss=11.6, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 88:  50%|███████▌       | 1/2 [00:14<00:14, 14.98s/it, loss=11.6, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 89:  50%|███████▌       | 1/2 [00:15<00:15, 15.14s/it, loss=11.6, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 90:  50%|███████▌       | 1/2 [00:15<00:15, 15.30s/it, loss=11.6, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 91:  50%|███████▌       | 1/2 [00:15<00:15, 15.46s/it, loss=11.6, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 92:  50%|███████▌       | 1/2 [00:15<00:15, 15.63s/it, loss=11.6, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 93:  50%|███████▌       | 1/2 [00:15<00:15, 15.79s/it, loss=11.6, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 94:  50%|███████▌       | 1/2 [00:15<00:15, 15.96s/it, loss=11.6, v_num=0]
Validation: 0it [00:00,

...

Epoch 114:  50%|███████       | 1/2 [00:18<00:18, 18.67s/it, loss=13.2, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 115:  50%|███████       | 1/2 [00:18<00:18, 18.83s/it, loss=13.2, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 116:  50%|███████       | 1/2 [00:18<00:18, 18.99s/it, loss=13.2, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 117:  50%|███████       | 1/2 [00:19<00:19, 19.15s/it, loss=13.2, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 118:  50%|███████       | 1/2 [00:19<00:19, 19.31s/it, loss=13.2, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 119:  50%|███████       | 1/2 [00:19<00:19, 19.52s/it, loss=13.2, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 120:  50%|███████       | 1/2 [00:19<00:19, 19.68s/it, loss=13.2, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 121:  50%|███████       | 1/2 [00:19<00:19, 19.85s/it, loss=13.2, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 122:  50%|███████       | 1/2 [00:20<00:20, 20.01s/it, loss=13.2, v_num=7]
Validation: 0it [00:00,

Epoch 143:  50%|███████       | 1/2 [00:23<00:23, 23.72s/it, loss=13.1, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 144:  50%|███████       | 1/2 [00:23<00:23, 23.91s/it, loss=13.1, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 145:  50%|███████       | 1/2 [00:24<00:24, 24.09s/it, loss=13.1, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 146:  50%|███████       | 1/2 [00:24<00:24, 24.27s/it, loss=13.1, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 147:  50%|███████       | 1/2 [00:24<00:24, 24.43s/it, loss=13.1, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 148:  50%|███████       | 1/2 [00:24<00:24, 24.59s/it, loss=13.1, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 149:  50%|███████       | 1/2 [00:24<00:24, 24.78s/it, loss=13.1, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 150:  50%|███████       | 1/2 [00:24<00:24, 24.96s/it, loss=13.1, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 151:  50%|███████       | 1/2 [00:25<00:25, 25.15s/it, loss=13.1, v_num=7]
Validation: 0it [00:00,

Epoch 171:  50%|███████       | 1/2 [00:28<00:28, 28.41s/it, loss=13.1, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 172:  50%|███████       | 1/2 [00:28<00:28, 28.58s/it, loss=13.1, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 173:  50%|███████       | 1/2 [00:28<00:28, 28.74s/it, loss=13.1, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 174:  50%|███████       | 1/2 [00:28<00:28, 28.90s/it, loss=13.1, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 175:  50%|███████       | 1/2 [00:29<00:29, 29.06s/it, loss=13.1, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 176:  50%|███████       | 1/2 [00:29<00:29, 29.22s/it, loss=13.1, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 177:  50%|███████       | 1/2 [00:29<00:29, 29.43s/it, loss=13.1, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 178:  50%|████████        | 1/2 [00:29<00:29, 29.59s/it, loss=13, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 179:  50%|████████        | 1/2 [00:29<00:29, 29.75s/it, loss=13, v_num=7]
Validation: 0it [00:00,

Epoch 200:  50%|████████        | 1/2 [00:33<00:33, 33.18s/it, loss=13, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 201:  50%|████████        | 1/2 [00:33<00:33, 33.34s/it, loss=13, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 202:  50%|████████        | 1/2 [00:33<00:33, 33.50s/it, loss=13, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 203:  50%|████████        | 1/2 [00:33<00:33, 33.66s/it, loss=13, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 204:  50%|████████        | 1/2 [00:33<00:33, 33.83s/it, loss=13, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 205:  50%|████████        | 1/2 [00:33<00:33, 33.99s/it, loss=13, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 206:  50%|████████        | 1/2 [00:34<00:34, 34.16s/it, loss=13, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 207:  50%|████████        | 1/2 [00:34<00:34, 34.32s/it, loss=13, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 208:  50%|████████        | 1/2 [00:34<00:34, 34.48s/it, loss=13, v_num=7]
Validation: 0it [00:00,

Epoch 228:  50%|███████       | 1/2 [00:37<00:37, 37.78s/it, loss=12.9, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 229:  50%|███████       | 1/2 [00:37<00:37, 37.95s/it, loss=12.9, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 230:  50%|███████       | 1/2 [00:38<00:38, 38.13s/it, loss=12.9, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 231:  50%|███████       | 1/2 [00:38<00:38, 38.30s/it, loss=12.9, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 232:  50%|███████       | 1/2 [00:38<00:38, 38.46s/it, loss=12.9, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 233:  50%|███████       | 1/2 [00:38<00:38, 38.62s/it, loss=12.9, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 234:  50%|███████       | 1/2 [00:38<00:38, 38.80s/it, loss=12.9, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 235:  50%|███████       | 1/2 [00:39<00:39, 39.01s/it, loss=12.9, v_num=7]
Validation: 0it [00:00, ?it/s]
Epoch 236:  50%|███████       | 1/2 [00:39<00:39, 39.19s/it, loss=12.9, v_num=7]
Validation: 0it [00:00,

In [ ]:
score = np.zeros(71)
for i in range (0,71):
    df = pd.read_csv("/home/luhung3080/Desktop/PycharmProjects/NCHUproject/Transformer/jupyterNB_logs/Transformer_csv/version_"+str(i)+"/metrics.csv")
    s=np.array(df['val_loss_MAE'].fillna(100))
    score[i]=np.min(s)

print(np.nanmean(score))

In [ ]:
m = np.nanmean(score)*np.ones(71)
fig1 = plt.figure(1)
x=np.linspace(1,71,71)
plt.plot(x,score, label = 'statione MAE')
plt.plot(x,m, label = 'mean MAE')
plt.legend(loc = 2)

plt.show()

In [ ]:
m = np.nanmean(score)*np.ones(71)
fig1 = plt.figure(1)
plt.bar(x,score,width=0.8)
# plt.xticks('stations')
plt.xlabel('stations')
plt.ylabel('MAE')
plt.plot(x,m, color='Orange')

plt.show()

In [ ]:
m = np.nanmean(score)*np.ones(71)
fig1 = plt.figure(1)
x=np.linspace(1,71,71)
plt.plot(x,score, label = 'statione MAE')
plt.plot(x,m, label = 'mean MAE')
plt.legend(loc = 2)

plt.show()

In [ ]:
m = np.nanmean(score)*np.ones(71)
fig1 = plt.figure(1)
plt.bar(x,score,width=0.8)
# plt.xticks('stations')
plt.xlabel('stations')
plt.ylabel('MAE')
plt.plot(x,m, color='Orange')

plt.show()